In [ ]:

# Data Processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Modelling
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [ ]:
ld_trans = pd.read_excel('Specific Districts HH Local Lite Duty Detail 2015 - YTD 2023_10 (2).xlsx',sheet_name='All Data')
util = pd.read_excel('Atlanta_West_15_19_Utilization.xlsx', sheet_name='Sheet1')

#subset to atlanta_west district
ld_trans = ld_trans[ld_trans['DISTRICT'] == '0030 - ATLANTA WEST FULTON     ']


ld_trans.reset_index(inplace=True,drop=True)

In [ ]:
#subset to all utilization
util = util[['Date', 'SumOfFLEET Gas', 'SumOfFLEET Parcel', 'SumOfFLEET All', 'Utilization Rate All']]

In [ ]:
ld_trans['DURATION'] = (ld_trans['DATE IN'] - ld_trans['DATE OUT']).dt.total_seconds()/(24*60*60)
ld_trans['MILEAGE PRICE'] = ld_trans['RATE MILE'] * ld_trans['MILES USED']
ld_trans['FUEL LOST'] = ld_trans['FUEL_OUT_LEVEL'] - ld_trans['FUEL_IN_LEVEL']
ld_trans['DAMAGED'] = ((ld_trans['DAMAGE_IN'] == 'Y') & (ld_trans['DAMAGE_OUT'] == 'N')).astype(int)

bins = np.arange(0, 11, 1)  # This creates an array [0, 1, 2, ..., 10]

ld_trans.loc[ld_trans['DURATION'] == 0, 'DURATION'] = 1

ld_trans['BOARD PRICE'] = ld_trans['RATE DAY']*ld_trans['DURATION']
ld_trans['EXP PRICE'] = ld_trans['BOARD PRICE'] + ld_trans['MILEAGE PRICE']

In [ ]:
cols = ['DATE OUT','TIME OUT','TIME IN','DATE IN','RATE DAY','RATE WEEK','RATE MILE','MILES USED','FUEL LOST','DAMAGED','DURATION','MILEAGE PRICE','BOARD PRICE','EXP PRICE','GROUP']


ld_trans = ld_trans[cols]

In [ ]:
start_date = ld_trans['DATE OUT'].min()
end_date = ld_trans['DATE IN'].max()

ld_trans.loc[:, 'Date'] = ld_trans['DATE OUT']

date_range = pd.date_range(start=start_date, end=end_date)

# Create a dataframe with this date range
df_base = pd.DataFrame(date_range, columns=['Date'])

# Left join other datraframes with this base dataframe
df_utilization_aligned = df_base.merge(util, on='Date', how='left')

# 3. Merge the Data
df_combined = ld_trans.merge(df_utilization_aligned, on='Date', how='left')

In [ ]:
def create_uhaul_price(current_date, reservation_date, duration, mileage):
    
    #get days reserved in advanced
    adv = (reservation_date - current_date).total_seconds() / (60*60*24)
    
    board_rate = 19.95 * duration
    mileage_rate = .59 if adv >= 2 else .69
    
    return board_rate + mileage_rate * mileage

In [ ]:
data = df_combined.dropna().drop('Date',axis=1)

mapping = {'LITE DUTY DIESEL': 1,
           'LITE DUTY GAS': 2,
           'PARCEL VAN-LITE DUTY': 3}

data['GROUP'] = data['GROUP'].replace(mapping)

data.reset_index(inplace=True,drop=True)
#data['UHAUL PRICE'] = [create_uhaul_price(data.loc[i,'RES CREATED'], data.loc[i,'DATE OUT'], data.loc[i,'DURATION'], data.loc[i,'MILES USED']) for i in range(len(data))]

In [ ]:
data.to_excel('Data_Atlanta_West.xlsx',index=False)